# Определение стоимости автомобилей

## Описание проекта

    Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля. 
    Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.
    Критерии, которые важны заказчику:
1. качество предсказания;
2. время обучения модели;
3. время предсказания модели.

In [1]:
!pip install catboost
!pip install lightgbm

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, OrdinalEncoder 
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMRegressor

## Загрузка данных

In [3]:
try:
    df = pd.read_csv('autos.csv', sep=',')
except:
    df = pd.read_csv('/datasets/autos.csv', sep=',')

In [4]:
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
# Приведем названия столбцов к змеиному регистру
df.columns = df.columns.str.lower()
df = df.rename(
    columns={'datecrawled': 'date_crawled', 'vehicletype': 'vehicle_type', 
             'registrationyear': 'registration_year', 'registrationmonth': 'registration_month', 
             'fueltype': 'fuel_type', 'datecreated': 'date_created', 
             'numberofpictures': 'number_of_pictures', 'postalcode': 'postal_code', 
             'lastseen': 'last_seen'})

Данные загружены, наименования столбцов переведены в змеиный регистр.

##  Изучите данные. Заполните пропущенные значения и обработайте аномалии в столбцах. Если среди признаков имеются неинформативные, удалите их.

In [6]:
# Просмотрим общую информацию о данных, проверим на пропуски и дубликаты
df.info()
print(df.isna().sum())
print('Явных дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  repaired            283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [7]:
df.describe()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [8]:
df['date_created'].sort_values(ascending=False).head(10)

323825    2016-04-07 00:00:00
238423    2016-04-07 00:00:00
218464    2016-04-07 00:00:00
8823      2016-04-07 00:00:00
136456    2016-04-07 00:00:00
115388    2016-04-07 00:00:00
124250    2016-04-07 00:00:00
262235    2016-04-07 00:00:00
140745    2016-04-07 00:00:00
84280     2016-04-07 00:00:00
Name: date_created, dtype: object

1. Обнаружены пропуски в столбцах vehicle_type, gearbox, model, fuel_type, repaired.
2. Обнаужены явные дубликаты.
3. В данных есть ненужные для обучения столбцы date_crawled, registration_month, date_created, last_seen, number_of_pictures, postal_code.
     'brand', 'model', '
4. Есть выбросы, например,в registration_year самый ранний год регистрации авто 1000 и самый поздний 9999, поэтому проверим дату последней регистрации объявления на сервисе - она равна 2016. 

1. Пропуски в gearbox и model удалим - бесполезные и невосстановимые строки, в которых отсутствуют почти все ключевые значения, в vehicle_type и  fuel_type заполним значением unknown, а repaired заполним yes, исходя из соображений, что если не указаны данные о ремонте, будем считать, что он был.
2. Дубликаты удалим.
3. Избавимся от ненужных столбцов, и объединим признаки 'brand' и 'model' в новый признак autos.
4. Отфильтруем объявления по году регистрции с 1980 по 2016 год, по цене выше 0, восстнановим нулевую мощность по признаку autos, используя медиану,а также ограничим мощность от 5 до 1000. 
5. Также приведем тип числовых данных к единому - int

In [9]:
df = df.drop_duplicates()
df.dropna(subset=['gearbox', 'model'], inplace = True) 
df['autos'] = df['brand'] + ' ' + df['model']
df = df.drop(
    ['date_crawled', 'registration_month', 'date_created', 'last_seen', 'number_of_pictures', 
     'brand', 'model', 'postal_code'], axis='columns'
)


In [10]:
df['vehicle_type'] = df['vehicle_type'].fillna('unknown')
df['fuel_type'] = df['fuel_type'].fillna('unknown')
df['repaired'] = df['repaired'].fillna('yes')

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 318958 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              318958 non-null  int64 
 1   vehicle_type       318958 non-null  object
 2   registration_year  318958 non-null  int64 
 3   gearbox            318958 non-null  object
 4   power              318958 non-null  int64 
 5   kilometer          318958 non-null  int64 
 6   fuel_type          318958 non-null  object
 7   repaired           318958 non-null  object
 8   autos              318958 non-null  object
dtypes: int64(4), object(5)
memory usage: 24.3+ MB


In [12]:
df = df[df['power'] < 1001]

In [13]:
for d in df['autos'].unique():
    df.loc[(df['autos'] == d) & (df['power'] == 0), 'power'] = \
    df.loc[(df['autos'] == d) & (df['power'] > 0), 'power'].median()

In [14]:
df = df[(df['price'] > 0) & (df['registration_year'] > 1979) & (df['registration_year'] < 2017) & (df['power'] > 4)].reset_index(drop=True)
df[['price', 'registration_year', 'power', 'kilometer']] = df[['price', 'registration_year', 'power', 'kilometer']].astype(int)


In [15]:
df.describe()

,price,registration_year,power,kilometer
count,298967.000000,298967.000000,298967.000000,298967.000000
mean,4740.376724,2003.011747,120.330458,128856.629661
std,4576.848103,5.900269,53.390488,36386.485736
min,1.000000,1980.000000,5.000000,5000.000000
25%,1290.000000,1999.000000,77.000000,125000.000000
50%,3000.000000,2003.000000,110.000000,150000.000000
75%,6900.000000,2007.000000,150.000000,150000.000000
max,20000.000000,2016.000000,1000.000000,150000.000000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298967 entries, 0 to 298966
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              298967 non-null  int32 
 1   vehicle_type       298967 non-null  object
 2   registration_year  298967 non-null  int32 
 3   gearbox            298967 non-null  object
 4   power              298967 non-null  int32 
 5   kilometer          298967 non-null  int32 
 6   fuel_type          298967 non-null  object
 7   repaired           298967 non-null  object
 8   autos              298967 non-null  object
dtypes: int32(4), object(5)
memory usage: 16.0+ MB


Пропущенные значения заполнены, аномалии в столбцах обработаны, неинформативные признаки удалены

## Подготовьте выборки для обучения моделей

In [17]:
# Проверка на мультиколлинеарность перед обучением
df.corr()

,price,registration_year,power,kilometer
price,1.000000,0.570655,0.502706,-0.385482
registration_year,0.570655,1.000000,0.132992,-0.362248
power,0.502706,0.132992,1.000000,0.096879
kilometer,-0.385482,-0.362248,0.096879,1.000000


Мультиколлинеарности нет, можно разделять выборки и обучать модели.
Обучать будем три модели для каждой из них подготовим признаки.

In [18]:
# Разделим на фичи и таргет, далее разделим на тренировачную и тестовую выборки
features = df.drop(['price'], axis=1)
target = df['price']
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.25, random_state=12345)
#Масштабируем признаки  
numeric = ['registration_year', 'power', 'kilometer']
pd.options.mode.chained_assignment = None
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Для LGBMRegressor категориальные признаки необходимо превести в тип category

In [19]:
# Зададим переменную с категориальными  признаками
categorical = ['vehicle_type', 'gearbox', 'fuel_type', 'repaired', 'autos']
# Скопируем фичи и изменим в них тип данных для модели
features_train_lgbm = features_train.copy()
features_test_lgbm = features_test.copy()
features_train_lgbm[categorical] = features_train[categorical].astype('category')
features_test_lgbm[categorical] = features_test[categorical].astype('category')

Для LinearRegression категориальные признаки нужно закодировать, используем для этого get_dummies

In [20]:
# Закодируем категориальные признаки 
#features_train_ohe = features_train.copy()
#features_test_ohe = features_test.copy()
#features_train_ohe = pd.get_dummies(features_train_ohe, drop_first=True)
#features_test_ohe = pd.get_dummies(features_test_ohe, drop_first=True)

In [21]:
df_ohe = df.copy()
df_ohe = pd.get_dummies(df_ohe, drop_first=True)
features_ohe = df_ohe.drop(['price'], axis=1)
target_ohe = df_ohe['price']
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(
features_ohe, target_ohe, test_size=0.25, random_state=12345)
#Масштабируем признаки  
numeric = ['registration_year', 'power', 'kilometer']
pd.options.mode.chained_assignment = None
scaler = StandardScaler()
scaler.fit(features_train_ohe[numeric])
features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])

In [22]:
#features_train_oe = features_train.copy()
#features_test_oe = features_test.copy()
# закодируем признаки с помощью порядкового кодирования
#enc = OrdinalEncoder()
#enc.fit(features_train_oe)
#features_train_oe = enc.transform(features_train_oe)
##enc.fit(features_test_oe)
#features_test_oe = enc.transform(features_test_oe)

Данные разделены и подготовлены для обучения

## Обучите разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробуйте разные гиперпараметры.

Обучим три модели: LGBMRegressor, CatBoostRegressor, LinearRegression.
1. Подберем гиперпараметры

In [23]:
# Строим LGBMRegressor

params_lgb = {
    'n_estimators': [i for i in range(50, 100)], 
    'learning_rate': [0.01, 0.1],
    'depth': [None] + [i for i in range(2, 12)],
    'num_leaves': [n for n in range(20, 200, 20)]
}

model = LGBMRegressor(random_state=12345)

model_lgb = RandomizedSearchCV(LGBMRegressor(random_state=12345), params_lgb, n_iter=10, scoring='neg_root_mean_squared_error', random_state=12345, n_jobs=-1, cv=3, verbose=10)

# Обучаем LGBMRegressor
model_lgb.fit(features_train_lgbm, target_train)
# Определяем лучяший результат и его гиперпараметры
display(model_lgb.best_params_)
display(model_lgb.best_score_*-1)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Warning] Unknown parameter: depth


{'num_leaves': 120, 'n_estimators': 97, 'learning_rate': 0.1, 'depth': 5}

1521.0809231969324

Лучшими для модели оказались параметры: num_leaves= 120, n_estimators= 97, learning_rate= 0.1, depth= 5, RMSE = 1521.08

In [24]:
# Строим CatBoostRegressor
params_cbr = {
    'learning_rate': [0.01, 0.1],
    'depth': [None] + [i for i in range(2, 12)],
    'n_estimators': [i for i in range(50, 100)]
}
model_cbr = RandomizedSearchCV(CatBoostRegressor(random_state=12345, cat_features=categorical), params_cbr, n_iter=10, scoring='neg_root_mean_squared_error', random_state=12345, n_jobs=-1, cv=3, verbose=10)

# Обучаем CatBoostRegressor
model_cbr.fit(features_train, target_train)
# Определяем лучяший результат и его гиперпараметры
display(model_cbr.best_params_)
display(model_cbr.best_score_*-1)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0:	learn: 4248.4173439	total: 286ms	remaining: 23.2s
1:	learn: 3954.3754840	total: 421ms	remaining: 16.8s
2:	learn: 3698.7157039	total: 588ms	remaining: 15.5s
3:	learn: 3471.3083070	total: 761ms	remaining: 14.8s
4:	learn: 3268.9852048	total: 916ms	remaining: 14.1s
5:	learn: 3094.0384550	total: 1.05s	remaining: 13.3s
6:	learn: 2941.0664686	total: 1.19s	remaining: 12.8s
7:	learn: 2809.9992427	total: 1.34s	remaining: 12.4s
8:	learn: 2692.1174392	total: 1.46s	remaining: 11.9s
9:	learn: 2591.8584787	total: 1.58s	remaining: 11.4s
10:	learn: 2504.9270439	total: 1.73s	remaining: 11.2s
11:	learn: 2428.4559999	total: 1.88s	remaining: 11s
12:	learn: 2362.1187575	total: 2.01s	remaining: 10.7s
13:	learn: 2306.0121746	total: 2.14s	remaining: 10.4s
14:	learn: 2259.7836982	total: 2.26s	remaining: 10.1s
15:	learn: 2216.6105990	total: 2.39s	remaining: 9.85s
16:	learn: 2181.3353665	total: 2.51s	remaining: 9.61s
17:	learn: 2147.6447555	total: 2.

{'n_estimators': 82, 'learning_rate': 0.1, 'depth': 9}

1712.160292687753

Лучшими для модели оказались параметры: n_estimators= 82, learning_rate= 0.1, depth= 9, RMSE = 1712.16


In [25]:
# Строим LinearRegression и обучаем
params_lr = {}
model_lr = RandomizedSearchCV(LinearRegression(), params_lr, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=12345, cv=3, verbose=10)
model_lr.fit(features_train_ohe, target_train_ohe)
display(model_lr.best_params_)
display(model_lr.best_score_*-1)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


{}

2408.866300912136

RMSE линейной модели = 2408,8



## Проанализируйте время обучения, время предсказания и качество моделей. 

Обучим модели на трейне с определенными гиперпараметрами и измерим время предсказания и обучения.

LGBMRegressor 

In [26]:
%%time

model_lgbm = LGBMRegressor(random_state=12345, num_leaves=120, n_estimators=97, learning_rate=0.1, depth=5)
model_lgbm.fit(features_train_lgbm, target_train)


Wall time: 1.51 s


LGBMRegressor(depth=5, n_estimators=97, num_leaves=120, random_state=12345)

In [27]:
%%time
predictions_lgbm = model_lgbm.predict(features_train_lgbm)
print('RMSE train lgbm:', mean_squared_error(target_train, predictions_lgbm)**0.5)


RMSE train lgbm: 1398.6915026842732
Wall time: 392 ms


Время обучения - 1.04 s, время предсказания - 306 ms. RMSE_train = 1398.69

CatBoostRegressor

In [28]:
%%time
model_cbr = CatBoostRegressor(
    random_state=12345, cat_features=categorical, n_estimators=82, learning_rate=0.1, depth=9
)
model_cbr.fit(features_train, target_train)


0:	learn: 4248.4173439	total: 136ms	remaining: 11.1s
1:	learn: 3954.3754840	total: 275ms	remaining: 11s
2:	learn: 3698.7157039	total: 437ms	remaining: 11.5s
3:	learn: 3471.3083070	total: 647ms	remaining: 12.6s
4:	learn: 3268.9852048	total: 847ms	remaining: 13s
5:	learn: 3094.0384550	total: 1.01s	remaining: 12.8s
6:	learn: 2941.0664686	total: 1.16s	remaining: 12.4s
7:	learn: 2809.9992427	total: 1.36s	remaining: 12.6s
8:	learn: 2692.1174392	total: 1.53s	remaining: 12.4s
9:	learn: 2591.8584787	total: 1.67s	remaining: 12s
10:	learn: 2504.9270439	total: 1.84s	remaining: 11.9s
11:	learn: 2428.4559999	total: 2.01s	remaining: 11.7s
12:	learn: 2362.1187575	total: 2.15s	remaining: 11.4s
13:	learn: 2306.0121746	total: 2.35s	remaining: 11.4s
14:	learn: 2259.7836982	total: 2.47s	remaining: 11.1s
15:	learn: 2216.6105990	total: 2.62s	remaining: 10.8s
16:	learn: 2181.3353665	total: 2.77s	remaining: 10.6s
17:	learn: 2147.6447555	total: 2.92s	remaining: 10.4s
18:	learn: 2105.9674899	total: 3.07s	remaini

In [29]:
%%time
predictions_cbr = model_cbr.predict(features_train)
print('RMSE train cbr:', mean_squared_error(target_train, predictions_cbr)**0.5)

RMSE train cbr: 1689.1114767319955
Wall time: 427 ms


Время обучения - 12.5 s, время предсказания - 429 ms. RMSE_train = 1689.11

LinearRegression

In [30]:
%%time
model_lr.fit(features_train_ohe, target_train_ohe)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Wall time: 18.4 s


RandomizedSearchCV(cv=3, estimator=LinearRegression(), n_jobs=-1,
                   param_distributions={}, random_state=12345,
                   scoring='neg_root_mean_squared_error', verbose=10)

In [31]:
%%time
predictions_lr = model_lr.predict(features_train_ohe)
print('RMSE train lr:', mean_squared_error(target_train_ohe, predictions_lr)**0.5)

RMSE train lr: 2403.283229286982
Wall time: 394 ms


Время обучения - 20.3 s, время предсказания - 435 ms. RMSE_train = 2403.28

Модели градинтного бустинга оказались фаворитами, однако LGBMRegressor показала лучшую скорость работы и лучшее значение RMSE.

## Опираясь на критерии заказчика, выберете лучшую модель, проверьте её качество на тестовой выборке. 

Проверим качество LGBMRegressor на тествовой выборке

In [32]:
%%time
predictions_lgbm_test = model_lgbm.predict(features_test_lgbm)
print('RMSE test lgbm:', mean_squared_error(target_test, predictions_lgbm_test)**0.5)

RMSE test lgbm: 1516.9358407534064
Wall time: 136 ms


RMSE на тестовой выборке лучшей модели = 1516,93

Полученные результаты оформим в таблицу.

In [33]:
table = pd.DataFrame({'Модель': ['LGBMRegressor', 'CatBoostRegressor', 'LinearRegression'], 
                      'время обучения, s': [1.04, 12.5, 20.3], 
                      'время предсказания, ms': [306, 429, 435], 
                      'RMSE': [1521, 1712.1, 2408.86] 
                     })
                     
table

,Модель,"время обучения, s","время предсказания, ms",RMSE
0,LGBMRegressor,1.04,306,1521.00
1,CatBoostRegressor,12.50,429,1712.10
2,LinearRegression,20.30,435,2408.86


Опираясь на результаты проведенной работы, заказчику рекомендуется использовать LGBMRegressor. Она показала самые лучшие результаты по RMSE и времени предсказания.

*Полученное время с каждым запуском кода изменяется, но не меняется фаворит.